In [1]:
import os
from pathlib import Path

import torch
import numpy as np
import pandas as pd
from torch import nn
from scipy.sparse import load_npz

os.sys.path.append('/hse_mlds_recsys_project')
from app.models.nn_models import NNModel, MatrixFactorization
from app.utils.metrics import precision_at_k

In [2]:
root_dir = Path('/hse_mlds_recsys_project/')

In [7]:
# class MatrixFactorization(torch.nn.Module):
#     def __init__(self, n_users, n_items, n_factors=150):
#         super().__init__()
#         self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
#         self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

#     def forward(self, user, item):
#         return (self.user_factors(user) * self.item_factors(item)).sum(1)


user_item_matrix = load_npz(root_dir / 'app' / 'data' / 'train_user_item_matrix.npz').astype('float')
n_users, n_items = user_item_matrix.shape

criterion = nn.MSELoss()
# net = MatrixFactorization(n_users, n_items, n_factors=150)
# optimizer = torch.optim.SGD(net.parameters(), lr=1e-6)

# rows, cols = user_item_matrix.nonzero()
# p = np.random.permutation(len(rows))
# rows, cols = rows[p], cols[p]

# for row, col in zip(rows, cols):
#     # Set gradients to zero
#     optimizer.zero_grad()
    
#     # Turn data into tensors
#     rating = torch.FloatTensor([user_item_matrix[row, col]])
#     row = torch.LongTensor([row])
#     col = torch.LongTensor([col])

#     # Predict and calculate loss
#     prediction = net(row, col)
#     loss = criterion(prediction, rating)

#     # Backpropagate
#     loss.backward()

#     # Update the parameters
#     optimizer.step()

In [3]:
! pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=3177cd51b2a8bc5a376be8f49f65a3e944b3bb269404c274dcb75b75b3b7de84
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [5]:
import wget

In [7]:
model_params_path = wget.download('https://docs.google.com/uc?export=download&id=1rcAthDY3_TDTlALK3MXKJJzFedcou6vU')
net_model = NNModel(MatrixFactorization, model_params_path, n_users=n_users, n_items=n_items)

Path(model_params_path).unlink()

In [22]:
%%time
net_model.get_recommendations('00055176fea33f6e027cd3302289378b')

CPU times: user 11.1 ms, sys: 2.27 ms, total: 13.4 ms
Wall time: 7.14 ms


['Frank Ocean__Pilot Jones',
 'Ciara__Goodies',
 'Franz Ferdinand__Do You Want To',
 'Madonna__Borderline',
 "Metronomy__I'm Aquarius"]

In [10]:
test = pd.read_csv('../datasets/test.csv.zip', compression='zip')
result = test.groupby('user_id')['song'].unique().reset_index().rename(columns={'song': 'actual'})

In [25]:
%%time
result['mf_150'] = result['user_id'].apply(lambda x: net_model.get_recommendations(x, n=20))
result.head()

CPU times: user 1min, sys: 142 ms, total: 1min
Wall time: 15.5 s


,user_id,actual,mf_150
0,00055176fea33f6e027cd3302289378b,"[Demi Lovato__Give Your Heart A Break, Ed Shee...","[Frank Ocean__Pilot Jones, Ciara__Goodies, Fra..."
1,0007f3dd09c91198371454c608d47f22,"[Ben Howard__Keep Your Head Up, Ed Sheeran__Sm...","[Muse__Panic Station, System Of A Down__Deer D..."
2,000b0f32b5739f052b9d40fcc5c41079,[Lars Winnerbäck__Om du lämnade mig nu],"[The Shins__Caring Is Creepy, The xx__Shelter,..."
3,000c11a16c89aa4b14b328080f5954ee,[Arctic Monkeys__Why'd You Only Call Me When Y...,"[Tina Turner__The Best, Daft Punk__Arrival, Me..."
4,00123e0f544dee3ab006aa7f1e5725a7,[Foster The People__Don't Stop (Color on the W...,"[Garbage__#1 Crush, Jessie J__Do It Like A Dud..."


In [29]:
result.apply(lambda row: precision_at_k(row['actual'], row['mf_150']), axis=1).mean()

0.0026603953847284486

### lr 1e-3

In [8]:
model_params_path = wget.download('https://docs.google.com/uc?export=download&id=1pkl7Iznw9xMc8ZzznIRFocYVK2Y1zoEC')
model_params = torch.load(model_params_path, map_location=torch.device('cpu'))
# net.load_state_dict(model_params['model_state'])
net_model = NNModel(MatrixFactorization, net_params_path=model_params_path, n_users=n_users, n_items=n_items)

Path(model_params_path).unlink()

In [11]:
%%time
result['mf_150_lr_003'] = result['user_id'].apply(lambda x: net_model.get_recommendations(x, n=20))
result.head()

CPU times: user 1min 37s, sys: 212 ms, total: 1min 38s
Wall time: 24.7 s


,user_id,actual,mf_150_lr_003
0,00055176fea33f6e027cd3302289378b,"[Demi Lovato__Give Your Heart A Break, Ed Shee...","[Noah And The Whale__L.I.F.E.G.O.E.S.O.N., Foa..."
1,0007f3dd09c91198371454c608d47f22,"[Ben Howard__Keep Your Head Up, Ed Sheeran__Sm...","[Billy Joel__My Life, The Black Eyed Peas__Hey..."
2,000b0f32b5739f052b9d40fcc5c41079,[Lars Winnerbäck__Om du lämnade mig nu],"[Death Cab for Cutie__The New Year, Rihanna__S..."
3,000c11a16c89aa4b14b328080f5954ee,[Arctic Monkeys__Why'd You Only Call Me When Y...,"[Pablo Alboran__Solamente Tú, James Taylor__Sw..."
4,00123e0f544dee3ab006aa7f1e5725a7,[Foster The People__Don't Stop (Color on the W...,"[Jake Bugg__A Song About Love, Goldfrapp__Ooh ..."


In [20]:
net_model.net.forward(torch.tensor(0), torch.arange(net_model.net.n_items)).argsort().flip(-1)

tensor([6319, 3070, 2443,  ...,   16,  868, 4246])

In [13]:
net_model.net.predict(0)

array([6319, 3070, 2443, ...,   16,  868, 4246])

In [35]:
%%time
result['mf_150_lr_003'] = result['user_id'].apply(lambda x: net_model.get_recommendations(x, n=20))
result.head()

CPU times: user 51.3 s, sys: 1.04 ms, total: 51.3 s
Wall time: 13 s


,user_id,actual,mf_biased_150_3_epoch,mf_150_lr_003
0,00055176fea33f6e027cd3302289378b,"[Demi Lovato__Give Your Heart A Break, Ed Shee...","[Tame Impala__She Just Won't Believe Me, The N...","[Noah And The Whale__L.I.F.E.G.O.E.S.O.N., Foa..."
1,0007f3dd09c91198371454c608d47f22,"[Ben Howard__Keep Your Head Up, Ed Sheeran__Sm...","[James Blunt__You're Beautiful, Candlebox__Far...","[Billy Joel__My Life, The Black Eyed Peas__Hey..."
2,000b0f32b5739f052b9d40fcc5c41079,[Lars Winnerbäck__Om du lämnade mig nu],"[Little Dragon__Little Man, Incubus__Nice to K...","[Death Cab for Cutie__The New Year, Rihanna__S..."
3,000c11a16c89aa4b14b328080f5954ee,[Arctic Monkeys__Why'd You Only Call Me When Y...,"[Metallica__Blackened, Gnarls Barkley__Smiley ...","[Pablo Alboran__Solamente Tú, James Taylor__Sw..."
4,00123e0f544dee3ab006aa7f1e5725a7,[Foster The People__Don't Stop (Color on the W...,"[Drake__All Me, Sharon Van Etten__Every Time t...","[Jake Bugg__A Song About Love, Goldfrapp__Ooh ..."


In [36]:
result.apply(lambda row: precision_at_k(row['actual'], row['mf_150_lr_003']), axis=1).mean()

0.002851473506283531

In [18]:
class BiasedMatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=150):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse=True)
        self.item_biases = torch.nn.Embedding(n_items, 1, sparse=True)

    def forward(self, user, item):
        pred = self.user_biases(user) + self.item_biases(item)
        pred += (
            (self.user_factors(user) * self.item_factors(item))
            .sum(dim=1, keepdim=True)
        )
        return pred.squeeze()

    def predict(self, user_index) -> np.array:
        with torch.inference_mode():
            pred = self.forward(
                user=torch.tensor([user_index]), 
                item=torch.arange(0, self.n_items)
            ).numpy()

        # sorting item scores descending
        return np.flip(pred.argsort())

In [5]:
torch.arange(0, 10).numpy()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [44]:
# net = BiasedMatrixFactorization(
#     n_users=n_users,
#     n_items=n_items,
#     n_factors=150
# )

In [8]:
model_params_path = wget.download('https://docs.google.com/uc?export=download&id=1CqmjqDLVcjKb6w300FFjL2_mDABiwvff')
# model_params = torch.load(model_params_path, map_location=torch.device('cpu'))
# net.load_state_dict(model_params['model_state'])
net_model = NNModel(BiasedMatrixFactorization, net_params_path=model_params_path, n_users=n_users, n_items=n_items)

Path(model_params_path).unlink()

In [10]:
%%time
result['mf_biased_150'] = result['user_id'].apply(lambda x: net_model.get_recommendations(x, n=20))
result.head()

CPU times: user 1min 46s, sys: 260 ms, total: 1min 47s
Wall time: 26.9 s


,user_id,actual,mf_biased_150
0,00055176fea33f6e027cd3302289378b,"[Demi Lovato__Give Your Heart A Break, Ed Shee...","[Linkin Park__Faint, Generationals__When They ..."
1,0007f3dd09c91198371454c608d47f22,"[Ben Howard__Keep Your Head Up, Ed Sheeran__Sm...","[Caribou__Back Home, Queen__We Are The Champio..."
2,000b0f32b5739f052b9d40fcc5c41079,[Lars Winnerbäck__Om du lämnade mig nu],"[Death From Above 1979__Trainwreck 1979, Van H..."
3,000c11a16c89aa4b14b328080f5954ee,[Arctic Monkeys__Why'd You Only Call Me When Y...,"[Ben Howard__Diamonds, Robbie Williams__Rock D..."
4,00123e0f544dee3ab006aa7f1e5725a7,[Foster The People__Don't Stop (Color on the W...,"[The Strokes__Under Control, Stereophonics__Ha..."


In [11]:
result.apply(lambda row: precision_at_k(row['actual'], row['mf_biased_150']), axis=1).mean()

0.002983758359667818

In [ ]:
data

In [14]:
cpt = torch.load(root_dir / 'app' / 'data' / 'matrix_factorization_bias_adagrad_3_epoch.pt',  map_location=torch.device('cpu'))

In [19]:
# model_params_path = wget.download('https://docs.google.com/uc?export=download&id=1AT5vCXQpCFsdBs9oF0HirLX5Cj6FK8ht')
net_model = NNModel(
    BiasedMatrixFactorization, 
    root_dir / 'app' / 'data' / 'matrix_factorization_bias_adagrad_3_epoch.pt', 
    n_users=n_users, 
    n_items=n_items
)

Path(model_params_path).unlink()

In [22]:
%%time
result['mf_biased_150_3_epoch'] = result['user_id'].apply(lambda x: net_model.get_recommendations(x, n=20))
result.head()

CPU times: user 1min 48s, sys: 325 ms, total: 1min 48s
Wall time: 27.6 s


,user_id,actual,mf_biased_150_3_epoch
0,00055176fea33f6e027cd3302289378b,"[Demi Lovato__Give Your Heart A Break, Ed Shee...","[Tame Impala__She Just Won't Believe Me, The N..."
1,0007f3dd09c91198371454c608d47f22,"[Ben Howard__Keep Your Head Up, Ed Sheeran__Sm...","[James Blunt__You're Beautiful, Candlebox__Far..."
2,000b0f32b5739f052b9d40fcc5c41079,[Lars Winnerbäck__Om du lämnade mig nu],"[Little Dragon__Little Man, Incubus__Nice to K..."
3,000c11a16c89aa4b14b328080f5954ee,[Arctic Monkeys__Why'd You Only Call Me When Y...,"[Metallica__Blackened, Gnarls Barkley__Smiley ..."
4,00123e0f544dee3ab006aa7f1e5725a7,[Foster The People__Don't Stop (Color on the W...,"[Drake__All Me, Sharon Van Etten__Every Time t..."


In [24]:
result.apply(lambda row: precision_at_k(row['actual'], row['mf_biased_150_3_epoch']), axis=1).mean()

0.0029396634085397224

In [41]:
adagrad_loss = torch.optim.Adagrad(model.parameters(), lr=1e-6)

In [45]:
len(rows), len(cols)

(1938057, 1938057)

In [ ]:
# rows, cols = user_item_matrix.nonzero()
# p = np.random.permutation(len(rows))
# rows, cols = rows[p], cols[p]

# for row, col in zip(rows, cols):
#     # Set gradients to zero
#     optimizer.zero_grad()
    
#     # Turn data into tensors
#     rating = torch.FloatTensor([user_item_matrix[row, col]])
#     row = torch.LongTensor([row])
#     col = torch.LongTensor([col])

#     # Predict and calculate loss
#     prediction = net(row, col)
#     loss = criterion(prediction, rating)

#     # Backpropagate
#     loss.backward()

#     # Update the parameters
#     optimizer.step()